# Asynchronous Algorithms

This exercise will introduce CUDA streams and the `AthAsynchronousAlgorithm` which allows us to run GPU code asynchronously (i.e. without blocking a CPU thread).
We will do this with a semi-contrived task calculating jet pull vectors using the CUB reduction algorithms mentioned last week.

The build of the code is done in the usual way.
  - On Perlmutter:

In [ ]:
!./scripts/run_on_perlmutter.sh cmake -DCMAKE_BUILD_TYPE=Debug -S . -B build
!./scripts/run_on_perlmutter.sh cmake --build build

  - On SWAN:

In [ ]:
!./scripts/run_on_swan.sh cmake -DCMAKE_BUILD_TYPE=Debug -S . -B build
!./scripts/run_on_swan.sh cmake --build build

Once the code's build is up to date, let's start by looking at the
[job config](CUDAExamples/python/03_Asynchronous.py), and noting the aspects
unique to a job with asynchronous algorithms (between the lines of hashes).

Then, we'll run it as-is.
  - On Perlmutter:

In [ ]:
!./scripts/run_on_perlmutter.sh ./build/CMakeFiles/atlas_build_run.sh athena.py --threads=2 --CA CUDAExamples/03_Asynchronous.py

  - On SWAN:

In [ ]:
!./scripts/run_on_swan.sh ./build/CMakeFiles/atlas_build_run.sh athena.py --threads=2 --CA CUDAExamples/03_Asynchronous.py

Next we'll take a quick look at the [header](CUDAExamples/src/03_Asynchronous/JetPullCUDAAlg.h) and [C++ source](CUDAExamples/src/03_Asynchronous/JetPullCUDAAlg.cxx) files of our algorithm, to see the special features we need to include:
* Inherit from `AthAsynchronousAlgorithm`, not `AthReentrantAlgorithm`
* Include some kind of `deviceExecute` function called from our `execute` function
  * In practice, this could be in a tool somewhere

Now let's move on to the meat of the code, the `deviceExecute` function and adjoining [CUDA code](CUDAExamples/src/03_Asynchronous/JetPullCUDAAlg.cu)
* Note the `Gaudi::CUDA::Stream` instantiated at the start. This is what we use to run multiple "streams" of CUDA code simultaneously. This is also what we use to determine when the fiber scheduler should wake us because the GPU is done with it's work.

## Task 1
As a first task, write the calls to copy the input data into the arrays in `d_jet` and `d_const`. For now, don't include the constituent count information.

You will be using the `cudaMemcpy` function with the following signature:
`cudaMemcpyAsync(dest, source, num_bytes, direction: cudaMemcpyHostToDevice, stream)`

If you get stuck, the solution is [here](CUDAExamples/src/03_Asynchronous/solution/JetPullCUDAAlg.cu.soln1).

In [ ]:
!./scripts/run_on_perlmutter.sh ./build/CMakeFiles/atlas_build_run.sh athena.py --threads=2 --CA CUDAExamples/03_Asynchronous.py

## Task 2
Your next task is to calculate the offsets into the array of constituents from the constituent count information.
You can do this using CUB's [DeviceScan](https://nvidia.github.io/cccl/cub/api/structcub_1_1DeviceScan.html) functionality.

First allocate the array to hold the information on the device and copy the input into it, as before. Use `cudaMemsetAlloc` to set the last element of the array to zero. It has the same signature as `cudaMemcpyAsync` but the second argument is a value, and it needs no direction. When you complete this task, you also need to uncomment the line that frees the `d_offsets` array you will be allocating.

Then, use `cub::DeviceScan::ExclusiveSum` to convert the counts into offsets. The key here is that you run it first with the `temp_storage` pointer set to a nullptr to determine how much storage to allocate (it returns immediately). You can then allocate the storage, and run it again to actually do the work. When it's done, remember to await the stream with `ATH_CHECK(stream.await());`, then free the temporary storage.

If you get stuck, the solution is [here](CUDAExamples/src/03_Asynchronous/solution/JetPullCUDAAlg.cu.soln2).

In [ ]:
!./scripts/run_on_perlmutter.sh ./build/CMakeFiles/atlas_build_run.sh athena.py --threads=2 --CA CUDAExamples/03_Asynchronous.py

## Task 3

Finally, we get to the heart of this exercise. You will write a kernel using [BlockReduce](https://nvidia.github.io/cccl/cub/api/classcub_1_1BlockReduce.html) to calculate the pulls, and also write the code to setup the output buffers, launch the kernel to run the calculation, then copy the output out to host memory, and finally free the output buffers.

If you get stuck, feel free to ask questions. If you get really stuck, the solution is [here](CUDAExamples/src/03_Asynchronous/solution/JetPullCUDAAlg.cu.soln3).

Note the kernel launch syntax:
```cuda
Kernels::calculatePulls<<<nJets, BLOCKSIZE, dynamic shared memory per stream: 0, stream>>>(...);
```

In your kernel you will use shared memory for the BlockReduce, but this is static, not dynamic. Also remember to call `__syncthreads()` before and after your block reductions.

### Jet Pull Definition

For the purpose of this task, the jet pull vector is defined using the following sum over constituents $c$ of jet $j$. The $\vec{r}$ vectors are in $[\eta, \phi]$ space.

$$\vec{p}_{j} = \sum_{c \in j} \left[ \frac{p^{c}_{T}}{p^{j}_{T}} \left|\vec{r}_{c} - \vec{r}_{j}\right| \left( \vec{r}_{c} - \vec{r}_{j} \right) \right]$$

In [ ]:
!./scripts/run_on_perlmutter.sh ./build/CMakeFiles/atlas_build_run.sh athena.py --threads=2 --CA CUDAExamples/03_Asynchronous.py

## Extensions for later
* In the example, we use `std::span`s for the arrays copied directly out of the event store. Would it be worth copying these into pinned memory first?
* How would you extend this to the correct definition of jet pull, using rapidities instead of pseudo-rapidities? Can you calculate the rapidities on the GPU?